In [38]:
import torch
import torch_geometric
import torch_geometric.nn as nn
import torch.nn.functional as F
import numpy as np

In [26]:
x = torch.randn(5, 2)

In [135]:
edge_index = np.random.randint(0, 5, (10, 2))
edge_matrix = torch.zeros((5, 5),dtype=torch.long)
for pos in edge_index:
    edge_matrix[pos[0], pos[1]] = 1
    edge_matrix[pos[1], pos[0]] = 1
for i in range(edge_matrix.shape[0]):
    edge_matrix[i][i] = 0
edge_index = np.concatenate([np.where(edge_matrix.numpy() == 1)], axis=1)
edge_index = torch.from_numpy(edge_index)
edge_index, edge_matrix

(tensor([[0, 0, 1, 1, 3, 3, 3, 4, 4, 4],
         [3, 4, 3, 4, 0, 1, 4, 0, 1, 3]]),
 tensor([[0, 0, 0, 1, 1],
         [0, 0, 0, 1, 1],
         [0, 0, 0, 0, 0],
         [1, 1, 0, 0, 1],
         [1, 1, 0, 1, 0]]))

In [136]:
#[nodes, input_dim] -> [nodes, output_dim]
gcnlayer = nn.GCNConv(2, 3)
api_out = gcnlayer(x, edge_index)
print("Api out: \n",api_out)

Api out: 
 tensor([[ 0.0818,  0.8151,  0.7033],
        [ 0.1497,  0.5080,  0.4334],
        [ 1.9875,  1.1867,  0.9308],
        [-0.0979,  0.3504,  0.3090],
        [-0.0979,  0.3504,  0.3090]], grad_fn=<AddBackward0>)


In [137]:
weight = gcnlayer.lin.weight.data
out = x @ weight.t() # [nodes, input_dim] -> [nodes, output_dim]
edge_matrix = torch.diag(torch.ones(edge_matrix.shape[0])) + edge_matrix #自连矩阵
deg = torch.sum(edge_matrix, dim=-1) 
deg = torch.sqrt(deg).unsqueeze(0) #度数开方
# 先聚合邻居信息，每个邻居的重要性用度数归一，然后每个节点用自身度数归一
out = edge_matrix / torch.tile(deg, (edge_matrix.shape[0], 1)) @ out / deg.t()
print("Out: \n", out)

Out: 
 tensor([[ 0.0818,  0.8151,  0.7033],
        [ 0.1497,  0.5080,  0.4334],
        [ 1.9875,  1.1867,  0.9308],
        [-0.0979,  0.3504,  0.3090],
        [-0.0979,  0.3504,  0.3090]])


tensor([[ 0.1433, -0.2548,  0.4093],
        [-0.4932,  0.0257, -0.5951],
        [-0.4949, -0.0991, -0.4778],
        [-0.4960, -0.2587, -0.3265],
        [-0.4949, -0.0991, -0.4778]])